In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv('input/bike/train.csv')

In [4]:
df.shape

(10886, 12)

In [8]:
df['year']=df.datetime.apply(lambda x:x.split('-')[0])
df['month']=df.datetime.apply(lambda x:x.split('-')[1])
df['date']=df.datetime.apply(lambda x:x.split('-')[2].split()[0])
df['time']=df.datetime.apply(lambda x:x.split()[1].split(':')[0])
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,date,time
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,01,01,00
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,01,01,01
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,01,01,02
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,01,01,03
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,01,01,04


In [10]:
X_date=df.iloc[:,-4:]

In [11]:
df=df.drop(X_date,axis=1)
df=df.drop(['datetime'],axis=1)
df.shape

(10886, 11)

In [12]:
df=X_date.join(df)

In [13]:
X_season=df.iloc[:,4].values
X_weather=df.iloc[:,7].values

In [9]:
y=df['count'].values
df=df.drop(['count','season','weather','casual','registered'],axis=1)

In [10]:
X_season=X_season.reshape(10886,1)
X_weather=X_weather.reshape(10886,1)

In [11]:
from sklearn.preprocessing import OneHotEncoder
ohe1=OneHotEncoder(categorical_features=[0])
ohe2=OneHotEncoder(categorical_features=[0])

In [12]:
X_season=ohe1.fit_transform(X_season).toarray()
X_weather=ohe2.fit_transform(X_weather).toarray()
X_season=X_season[:,1:]
X_weather=X_weather[:,1:]

/anaconda3/envs/digiml/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/envs/digiml/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/anaconda3/envs/digiml/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWar

In [13]:
X_sw=np.append(X_season,X_weather,axis=1)

In [14]:
sw=pd.DataFrame(X_sw,columns=['season_1','season_2','season_3','weather_1','weather_2','weather_3'])

In [15]:
X=df.join(sw,how='right')

In [16]:
X=X.values

In [17]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X=sc.fit_transform(X)

In [18]:
# Splitting the Data
from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,random_state=0,test_size=0.10)

In [19]:
# Fitting the Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
regressor=RandomForestRegressor(n_estimators=75,random_state=0)
regressor.fit(Xtrain,ytrain)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=75,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [20]:
regressor.score(Xtest,ytest)

0.9456080481277117

In [21]:
y_pred=regressor.predict(Xtest)

In [22]:
report=pd.DataFrame(np.array([ytest,y_pred]).T,columns=['original','prediction'])

In [23]:
report

,original,prediction
0,244.0,251.680000
1,239.0,315.066667
2,229.0,236.586667
3,467.0,577.706667
4,335.0,414.613333
5,40.0,60.080000
6,329.0,354.133333
7,2.0,2.786667
8,141.0,166.626667
9,391.0,445.226667


In [24]:
import model_utils as mdu
mdu.save_model(regressor,file='bike_sharing_model.p')

True

In [25]:
mdu.save_model(ohe1,file='bike_sharing_ohe1.p')
mdu.save_model(ohe2,file='bike_sharing_ohe2.p')
mdu.save_model(sc,file='bike_sharing_scaler.p')

True